In [1]:
import duckdb
import rasterio
import os
from keplergl import KeplerGl
import pandas as pd
import shapely
from rasterstats import zonal_stats, point_query
from concurrent.futures import ThreadPoolExecutor

In [2]:
RES = 8 # can go till 11 but after we will have <2 samples per cell I think
OVERTURE_VERSION = "2025-08-20.0"

In [3]:
con = duckdb.connect()
con.install_extension("spatial")
con.load_extension("spatial")

con.install_extension("h3", repository="community")
con.load_extension("h3")


In [4]:
if not os.path.exists("overture_country_pk.parquet"):
    con.sql(f"""
        COPY(
        SELECT
            *
        FROM
            read_parquet('s3://overturemaps-us-west-2/release/{OVERTURE_VERSION}/theme=divisions/type=division_area/*', hive_partitioning=1)
        WHERE
            subtype = 'country'
            AND country = 'PK'
        ) TO 'overture_country_pk.parquet'
    """)

In [5]:
# Load the parquet file into a pandas DataFrame
df = con.sql("SELECT *, ST_AsText(geometry) as geom FROM 'overture_country_pk.parquet'").df()

# Visualize in KeplerGl
map_1 = KeplerGl(height=800)
map_1.add_data(data=df[["geom"]], name="PK Country")


User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


In [6]:
# Download the GLO-30 hand geojson file
!aws s3 cp --no-sign-request s3://glo-30-hand/v1/2021/glo-30-hand.geojson ./

download: s3://glo-30-hand/v1/2021/glo-30-hand.geojson to ./glo-30-hand.geojson


In [7]:
rois_hand = con.sql("""
    With hand_raster_polys as (
    SELECT UNNEST(features) as feature,
    st_geomfromgeojson(json_extract_string(feature, '$.geometry')) AS geom,
    json_extract_string(feature, '$.properties.file_path') as file_path
    FROM read_json_auto('glo-30-hand.geojson')
    )
    SELECT DISTINCT 
        geom, file_path  
    FROM hand_raster_polys, read_parquet('overture_country_pk_with_JK.parquet')
    WHERE ST_Intersects(geom, geometry)
        
""")
rois_hand.show()

┌─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┬───────────────────────────────────────────────────────────────────────────────────────────────────────────────┐
│                                                        geom                                                         │                                                   file_path                                                   │
│                                                      geometry                                                       │                                                    varchar                                                    │
├─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┼───────────────────────────────────────────────────────────────────────────────────────────────────────────────┤
│ POLYGON ((73.999861 35.000139, 73.999861 34.000139, 74.999861 34.00013

In [8]:
# Load the parquet file into a pandas DataFrame
df = rois_hand.select("ST_AsText(geom) as geom,file_path").df()

# Visualize in KeplerGl
map_2 = KeplerGl(height=800)
map_2.add_data(data=df[["geom", "file_path"]], name="PK Country")
map_2

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


KeplerGl(data={'PK Country': {'index': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, …

In [9]:
os.makedirs("hand_cogs", exist_ok=True)
import urllib.request
import tqdm
from concurrent.futures import ThreadPoolExecutor


file_paths = rois_hand.select("file_path").df()["file_path"]
def download_file(url):
    filename = os.path.join("hand_cogs", os.path.basename(url.split("/")[-1]))
    if not os.path.exists(filename):
        print(f"Downloading {url} to {filename}")
        urllib.request.urlretrieve(url.replace("/vsicurl/", ""), filename)

with ThreadPoolExecutor() as executor:
    list(tqdm.tqdm(executor.map(download_file, file_paths), total=len(file_paths)))

100%|██████████| 131/131 [00:00<00:00, 582665.77it/s]


In [10]:
bboxes = con.read_parquet('overture_country_pk.parquet')['bbox'].select("bbox.xmin", "bbox.ymin", "bbox.xmax", "bbox.ymax").df()
minx, miny, maxx, maxy = min(bboxes["xmin"]), min(bboxes["ymin"]), max(bboxes["xmax"]), max(bboxes["ymax"])

In [11]:
rois_hand_df = rois_hand.select(
    duckdb.SQLExpression("ST_AsText(geom)").alias("geom_wkt"),'file_path').df()

In [12]:
if not os.path.exists("buildings_pk.parquet"):
    con.sql(f"""
        COPY (
            SELECT
                *
            FROM read_parquet('s3://overturemaps-us-west-2/release/{OVERTURE_VERSION}/theme=buildings/type=*/*', hive_partitioning=1, union_by_name=True)
        WHERE
            bbox.xmin > {minx}
            AND bbox.xmax < {maxx}
            AND bbox.ymin > {miny}
            AND bbox.ymax < {maxy}
    ) TO 'buildings_pk.parquet'
    """
    )

In [ ]:
buildings = con.read_parquet('buildings_pk.parquet')

for row in tqdm.tqdm(rois_hand_df.values):
    os.makedirs(f"building_density_at_res_{RES}", exist_ok=True)
    geom_wkt = row[0]
    file_path_from_json = os.path.join("hand_cogs", os.path.basename(row[1].split("/")[-1]))
    file_path = f"building_density_at_res_{RES}/building_density_{os.path.splitext(os.path.basename(file_path_from_json))[0]}_{RES}.parquet"
    if not os.path.exists(file_path):
        building_rois = buildings.filter(f"ST_Intersects(geometry, ST_GeomFromText('{geom_wkt}'))")
        building_density = (building_rois
            .select(duckdb.SQLExpression("ST_Centroid(geometry)").alias("centroid"))
            .select(duckdb.SQLExpression(f"h3_latlng_to_cell_string(ST_Y(centroid), ST_X(centroid), {RES})").alias("h3_cell"))
        )

        building_density = con.sql("""
                SELECT 
                        h3_cell, 
                        COUNT(*) as building_count,
                        -- h3_cell_to_boundary_wkt(h3_cell) as building_boundary,
                        ST_AsText(ST_POINT(h3_cell_to_lng(h3_cell), h3_cell_to_lat(h3_cell))) as centroid_wkt
                FROM building_density GROUP BY h3_cell
        """)
        building_density_df = building_density.df()
        building_density_df["centroid"] = building_density_df["centroid_wkt"].apply(shapely.wkt.loads)

        def query_points_batch(geoms):
            return point_query(geoms, file_path_from_json)

        batch_size = 1000
        results = []
        with ThreadPoolExecutor() as executor:
            batches = [building_density_df["centroid"][i:i+batch_size] for i in range(0, len(building_density_df), batch_size)]
            batch_results = list(executor.map(query_points_batch, batches))
            for res in batch_results:
                results.extend(res)

        building_density_df["point_query"] = results
        building_density_df.drop("centroid", axis=1).to_parquet(file_path, index=False)


100%|██████████| 131/131 [00:00<00:00, 92923.02it/s]


In [14]:
# Use DuckDB to read and concatenate all parquet files and write to a single parquet file
if not os.path.exists(f'merged_res_{RES}.parquet'):
    con.sql(f"""
        COPY (SELECT * FROM read_parquet('building_density_at_res_{RES}/building_density_*.parquet', union_by_name=True)) 
        TO 'merged_res_{RES}.parquet' (FORMAT 'parquet')
    """)

In [15]:
map_config = {'version': 'v1',
 'config': {'visState': {'filters': [],
   'layers': [{'id': 'pk7kxsk',
     'type': 'hexagonId',
     'config': {'dataId': 'building density',
      'label': 'building density',
      'color': [34, 63, 154],
      'highlightColor': [252, 242, 26, 255],
      'columns': {'hex_id': 'h3_cell'},
      'isVisible': True,
      'visConfig': {'colorRange': {'name': 'Global Warming',
        'type': 'sequential',
        'category': 'Uber',
        'colors': ['#FFC300',
         '#F1920E',
         '#E3611C',
         '#C70039',
         '#900C3F',
         '#5A1846'],
        'reversed': True},
       'filled': True,
       'opacity': 0.8,
       'outline': False,
       'strokeColor': None,
       'strokeColorRange': {'name': 'Global Warming',
        'type': 'sequential',
        'category': 'Uber',
        'colors': ['#5A1846',
         '#900C3F',
         '#C70039',
         '#E3611C',
         '#F1920E',
         '#FFC300']},
       'strokeOpacity': 0.8,
       'thickness': 2,
       'coverage': 1,
       'enable3d': True,
       'sizeRange': [0, 500],
       'coverageRange': [0, 1],
       'elevationScale': 5,
       'enableElevationZoomFactor': True},
      'hidden': False,
      'textLabel': [{'field': None,
        'color': [255, 255, 255],
        'size': 18,
        'offset': [0, 0],
        'anchor': 'middle',
        'alignment': 'center',
        'outlineWidth': 0,
        'outlineColor': [255, 0, 0, 255],
        'background': False,
        'backgroundColor': [0, 0, 200, 255]}]},
     'visualChannels': {'colorField': {'name': 'point_query', 'type': 'real'},
      'colorScale': 'quantile',
      'strokeColorField': None,
      'strokeColorScale': 'quantile',
      'sizeField': {'name': 'building_count', 'type': 'integer'},
      'sizeScale': 'linear',
      'coverageField': None,
      'coverageScale': 'linear'}}],
   'effects': [],
   'interactionConfig': {'tooltip': {'fieldsToShow': {'building density': [{'name': 'h3_cell',
        'format': None},
       {'name': 'building_count', 'format': None},
       {'name': 'point_query', 'format': None}]},
     'compareMode': False,
     'compareType': 'absolute',
     'enabled': True},
    'brush': {'size': 0.5, 'enabled': False},
    'geocoder': {'enabled': False},
    'coordinate': {'enabled': False}},
   'layerBlending': 'normal',
   'overlayBlending': 'normal',
   'splitMaps': [],
   'animationConfig': {'currentTime': None, 'speed': 1},
   'editor': {'features': [], 'visible': True}},
  'mapState': {'bearing': 24,
   'dragRotate': True,
   'latitude': 30.531893616040605,
   'longitude': 70.01893558941872,
   'pitch': 50,
   'zoom': 6.795093380133405,
   'isSplit': False,
   'isViewportSynced': True,
   'isZoomLocked': False,
   'splitMapViewports': []},
  'mapStyle': {'styleType': 'dark-matter',
   'topLayerGroups': {},
   'visibleLayerGroups': {'label': True,
    'road': True,
    'border': False,
    'building': True,
    'water': True,
    'land': True,
    '3d building': False},
   'threeDBuildingColor': [15.035172933000911,
    15.035172933000911,
    15.035172933000911],
   'backgroundColor': [0, 0, 0],
   'mapStyles': {}}}}

In [16]:
# this will blow at higher resolutions - for those its better to load individual parquet files then merged ones
map_4 = KeplerGl(height=600)
map_4.add_data(data=con.read_parquet(f'merged_res_{RES}.parquet').df(), name="building density")
map_4.config = map_config
map_4

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


/Users/shabbir/.pyenv/versions/3.11.13/envs/env_analysis/lib/python3.11/site-packages/jupyter_client/session.py:721: UserWarning: Message serialization failed with:
Out of range float values are not JSON compliant
Supporting this message is deprecated in jupyter-client 7, please make sure your message is JSON-compliant
  content = self.pack(content)


KeplerGl(config={'version': 'v1', 'config': {'visState': {'filters': [], 'layers': [{'id': 'pk7kxsk', 'type': …

In [17]:
map_4.save_to_html(file_name=f"map_at_res_{RES}.html")

Map saved to map_at_res_8.html!
